In [ ]:
import json, os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
INPUT_TXT  = "Test_tweets.txt"   # one tweet per line, for testing
BATCH_SIZE = 20
MODEL_NAME = "gpt-5-mini"

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def extract_batch(tweets: list[str]) -> list[dict]:
    prompt = """
You are analyzing Turkish tweets.
For each tweet, return ONLY a JSON array with one object per tweet:
- "location": city/district/neighborhood if present; else null
- "request": the item being asked for (exact Turkish word/phrase from the tweet, e.g. "çadır", "ekmek", "vinç")
- "amount": integer if a number is present; else null

No explanations, only the JSON array.
"""
    numbered = "\n".join(f"{i+1}. {t}" for i, t in enumerate(tweets))
    full = prompt + "\nTweets:\n" + numbered

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": full}],
    )
    content = resp.choices[0].message.content

    try:
        data = json.loads(content)
        if isinstance(data, list) and len(data) == len(tweets):
            return data
        
    except Exception as e:
        print(e)
    return [{"location": None, "request": None, "amount": None} for _ in tweets]

def keep_row(info: dict) -> bool:
    return bool((info.get("location") or "").strip()) and bool((info.get("request") or "").strip())

def main():
    with open(INPUT_TXT, "r", encoding="utf-8") as f:
        tweets = [line.strip() for line in f if line.strip()]

    for i in range(0, len(tweets), BATCH_SIZE):
        batch = tweets[i:i+BATCH_SIZE]
        preds = extract_batch(batch)
        for tweet, info in zip(batch, preds):
            if keep_row(info):
                amount = info.get("amount") or 1 
                request = (info.get("request") or "").lower().strip()
                print("Tweet:   ", tweet)
                print("Location:", info.get("location"))
                print("Request: ", request)
                print("Amount:  ", amount)
        print(f"Processed {i+len(batch)} / {len(tweets)}\n")

if __name__ == "__main__":
    main()


Tweet:    solcugazete adıyaman merkezde 1 tane çadır ihtiyacımız var evde duramıyoruz
Location: adıyaman merkez
Request:  çadır
Amount:   1
Tweet:    hataysamandağ acilçadır hataydefne acilçadıryardımı hatayaacil çadıryardımı
Location: hatay samandağ
Request:  çadır
Amount:   1
Tweet:    serkantopalchp adıyaman merkezde 1 tane çadır ihtiyacımız var evde duramıyoruz
Location: adıyaman merkez
Request:  çadır
Amount:   1
Tweet:    hataysamandağ acilçadır hataydefne acilçadıryardımı hatayaacil çadıryardımı
Location: hatay samandağ
Request:  çadır
Amount:   1
Tweet:    mhsnkrmzgl adıyaman merkezde 1 tane çadır ihtiyacımız var evde duramıyoruz
Location: adıyaman merkez
Request:  çadır
Amount:   1
Tweet:    adıyaman merkezde 1 tane çadır ihtiyacımız var evde duramıyoruz
Location: adıyaman merkez
Request:  çadır
Amount:   1
Tweet:    okulumuzun ogretmenlerinden mustafa akbayranin köyüne yardım gerekiyorlutfen rt yapalım gaziantep yavuzeli akbayır evleri girilemeyecek olduğu halde şu anda ihtiy